# Training and Evaluation

In [1]:
# system imports
import os

# additional imports
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import torch

# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyperparameters

In [2]:
hparams = {    
    "epochs": 20,
    "batch_size": 8,
    "lr": 1e-3,
    "features": [
        'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate',
        'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 
        'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20'
    ]
}

## Prepare Data

In [3]:
df_features = pd.read_csv("data/prepared_data.csv")
X = np.array(df_features[hparams['features']], dtype=np.float32)

encoder = LabelEncoder()
y = encoder.fit_transform(df_features['label'])
print("classes:", encoder.classes_)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)

# create pytorch dataloader
torch.manual_seed(42)
train_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train).long())
test_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test).long())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=hparams["batch_size"], shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=hparams["batch_size"], shuffle=False)

classes: ['covid' 'not_covid']
X_train.shape: (113, 26)
y_train.shape: (113,)


## Setup Model

In [4]:
# Design model (input, output size, forward pass)
class CoughNet(torch.nn.Module):
    def __init__(self, input_size):
        super(CoughNet, self).__init__()
        self.l1 = torch.nn.Linear(input_size, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)
        self.l6 = torch.nn.Linear(10, 2)

    def forward(self, x):
        x = torch.relu(self.l1(x))
        x = torch.relu(self.l2(x))
        x = torch.relu(self.l3(x))
        x = torch.relu(self.l4(x))
        x = torch.relu(self.l5(x))
        x = self.l6(x)
        return x

model = CoughNet(len(hparams["features"])).to(device)

## Training

In [5]:
# Construct loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=hparams["lr"])
criterion = torch.nn.CrossEntropyLoss()

def train(loader_train, model, optimizer, epoch):
    model.train()
    running_loss = 0.0
    running_correct = 0.0
    total = 0
    pbar = tqdm(enumerate(loader_train), total=len(loader_train))
    for batch_ndx, sample in pbar: 
        features, labels = sample[0].to(device), sample[1].to(device) 

        # forward pass and loss calculation
        outputs = model(features)
        loss = criterion(outputs, labels)  
        
        # backward pass    
        loss.backward()
        
        # update weights
        optimizer.step()
        optimizer.zero_grad()
        
        # calculate metrics
        running_loss += loss.item()
        predictions = torch.argmax(outputs.data, 1)
        running_correct += (predictions == labels).sum().item()

        # print informations
        pbar.set_description(f"[Training Epoch {epoch+1}]") 
        total += labels.shape[0]
        pbar.set_postfix({'loss': running_loss / total, 'train_accuracy': running_correct / total})

def evaluate(loader_test, model, epoch):
    model.eval()
    with torch.no_grad():
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        pbar = tqdm(enumerate(loader_test), total=len(loader_test))
        for batch_ndx, sample in pbar:
            features, labels = sample[0].to(device), sample[1].to(device) 

            # forward pass and loss calculation
            outputs = model(features)
            loss = criterion(outputs, labels)  

            # calculate metrics
            running_loss += loss.item()
            predictions = torch.argmax(outputs.data, 1)
            running_correct += (predictions == labels).sum().item()

            # print informations
            pbar.set_description(f"[Evaluating Epoch {epoch+1}]")
            total += labels.shape[0]
            pbar.set_postfix({'loss': running_loss / total, 'eval_accuracy': running_correct / total})

# training loop
for epoch in range(hparams["epochs"]):
    train(train_loader, model, optimizer, epoch)
    evaluate(test_loader, model, epoch)

# save checkpoint after training
checkpoint = {
    'hparams': hparams,
    'model_state': model.state_dict(),
    'scaler': scaler,
    'encoder': encoder
}
torch.save(checkpoint, "checkpoints/checkpoint.pth")

[Evaluating Epoch 20]: 100%|██████████| 8/8 [00:00<00:00, 201.13it/s, loss=0.296, eval_accuracy=0.947]
